In [37]:
import pandas as pd
import numpy as np

In [39]:
BASE = 'afastamentos'
#BASE = 'cadastro'
df = pd.read_csv(BASE + '-nomes-sexo.csv', index_col=0)

In [40]:
df.rename(columns={'sexo': 'SEXO'}, inplace=True)

In [41]:
df.head(10)

,SEXO
PNOME,
MARIA,F
JOSE,M
ANTONIO,M
FRANCISCO,M
JOAO,M
CARLOS,M
ANA,F
PAULO,M
LUIZ,M


In [83]:
df[['SEXO', 'REV']].groupby(by=['SEXO']).count().rename({'REV': 'QTDE'}, axis=1)

,QTDE
SEXO,
0,1015
1,4376
2,4335


In [42]:
df['SEXO'] = df['SEXO'].map({'F': 1, 'M': 2, 'X': 0}).astype('uint8')

In [43]:
df['REV'] = df.index
df['REV'] = df['REV'].apply(lambda x: x[::-1])

In [44]:
df.head()

,SEXO,REV
PNOME,,
MARIA,1,AIRAM
JOSE,2,ESOJ
ANTONIO,2,OINOTNA
FRANCISCO,2,OCSICNARF
JOAO,2,OAOJ


In [45]:
MAIOR_QTDE_LETRAS = 16 # fixado para o maior arquivo

for i in range(MAIOR_QTDE_LETRAS):
    df['L' + str(i)] = df['REV'].apply(
        lambda x: ord(x[i]) - 64 if len(x) > i else 0
    ).astype('uint8')

In [46]:
#df.dtypes

In [47]:
df.head(10)

,SEXO,REV,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,,
MARIA,1,AIRAM,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,2,ESOJ,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,2,OINOTNA,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,2,OCSICNARF,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,2,OAOJ,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,2,SOLRAC,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0
ANA,1,ANA,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
PAULO,2,OLUAP,15,12,21,1,16,0,0,0,0,0,0,0,0,0,0,0
LUIZ,2,ZIUL,26,9,21,12,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
#df.describe()

In [88]:
# definir dados de entrada

X = df.drop(['SEXO', 'REV'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo

print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (9726, 16) (9726,)


In [89]:
# dividir dados em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Shapes: (6516, 16) (3210, 16) (6516,) (3210,)


In [54]:
# treinar com todos os dados disponíveis

X_train, y_train = X, y

print("Shapes:", X_train.shape, y_train.shape)

Shapes: (9726, 4) (9726,)


https://automl.github.io/auto-sklearn/master/manual.html

In [73]:
# https://automl.github.io/auto-sklearn/master/installation.html

#!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip3 install
#!pip3 install auto-sklearn

In [63]:
import autosklearn.classification

In [64]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=10*60,
    per_run_time_limit=2*60,
    n_jobs=2,
    memory_limit=1024,
    seed=42,
    include_preprocessors=["no_preprocessing"]
)
automl.fit(X_train, y_train)

AutoSklearnClassifier(include_preprocessors=['no_preprocessing'],
                      memory_limit=1024, n_jobs=2, per_run_time_limit=120,
                      seed=42, time_left_for_this_task=600)

In [65]:
print(automl.show_models())

[(0.280000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'qda', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'minmax', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:qda:reg_param': 0.6019930535728735},
dataset_properties={
  'task': 2,
  'sparse': False,
  'multilabel': False,
  'multiclass': True,
  'target_type': 'classification',
  'signed': False})),
(0.100000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'extra_trees', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority

In [66]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: 10ec9315-9b96-11eb-88de-43abf76a2574
  Metric: accuracy
  Best validation score: 0.853271
  Number of target algorithm runs: 35
  Number of successful target algorithm runs: 31
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0



In [68]:
losses_and_configurations = [
    (run_value.cost, run_key.config_id)
    for run_key, run_value in automl.automl_.runhistory_.data.items()
]
losses_and_configurations.sort()
print("Lowest loss:", losses_and_configurations[0][0])
print(
    "Best configuration:",
    automl.automl_.runhistory_.ids_config[losses_and_configurations[0][1]]
)

Lowest loss: 0.1467289719626168
Best configuration: Configuration:
  balancing:strategy, Value: 'none'
  classifier:__choice__, Value: 'libsvm_svc'
  classifier:libsvm_svc:C, Value: 1.171444369812126
  classifier:libsvm_svc:gamma, Value: 0.016364556691954846
  classifier:libsvm_svc:kernel, Value: 'rbf'
  classifier:libsvm_svc:max_iter, Constant: -1
  classifier:libsvm_svc:shrinking, Value: 'True'
  classifier:libsvm_svc:tol, Value: 1.2474972913820311e-05
  data_preprocessing:categorical_transformer:categorical_encoding:__choice__, Value: 'no_encoding'
  data_preprocessing:categorical_transformer:category_coalescence:__choice__, Value: 'no_coalescense'
  data_preprocessing:numerical_transformer:imputation:strategy, Value: 'median'
  data_preprocessing:numerical_transformer:rescaling:__choice__, Value: 'none'
  feature_preprocessor:__choice__, Value: 'no_preprocessing'



In [58]:
# carregar dados de teste

In [69]:
y_pred = automl.predict(X_test)

In [70]:
from sklearn.metrics import accuracy_score

print("Accuracy score: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

Accuracy score: 91.78 %


In [72]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

Confusion matrix:
 [[ 158   61   88]
 [  13 1360   66]
 [   6   30 1428]]
